In [1]:
import pandas as pd 
import numpy as np 
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import plotly.express as px
import sys 

In [35]:
from data_upload import data_pretreatment
df = pd.read_csv("G:\生生不息\实验数据\BaiduSyncdisk\dataframe4.csv")
data_pretreatment(4,df)

plants = pd.read_csv("G:\生生不息\实验数据\BaiduSyncdisk\Plant_data_halfhour.csv")
whole_time=[pd.to_datetime(day+' '+hour) for day,hour in zip(plants['Date'],plants['Time'])]
plants['time']=whole_time
plants['pixel diff']=plants['Area per plant'].diff(1)
plants=plants.set_index('time')
plants['time'] = plants.index
#plants = plants[:-1]
re_arranged_plants = pd.DataFrame({})
df_collection = []

for i in range(1,10):
    new_df = pd.DataFrame({})
    new = []
    times = []
    #print(i)
    for j in range(len(plants)):
        if plants['No.'][j] == i:
            new.append(plants['Area per plant'][j])
            times.append(plants['time'][j])
    new_df['plant '+str(i)] = new
    new_df['plant '+str(i)+' diff'] = new_df['plant '+str(i)].diff(1)
    new_df['time'] = times 
    new_df['time'] = pd.to_datetime(new_df['time'])
    #new_df = new_df.set_index('time')
    #new_df.index = pd.to_datetime(new_df.index)
    #ew_df = new_df.resample('30m').mean()
    try:
        #new_df.resample('1H').mean()
        df_collection.append(new_df)
    except:
        pass
from glob import glob
import os 

dataframes4 = {}
csv_path = "arranged_data4"
files = os.listdir(csv_path)
num_files = len(files)

for i in range(num_files):
    dataframes4['df'+str(i)] = pd.read_csv("./arranged_data4/sub_df"+str(i)+".csv")

combine_df = pd.concat([dataframes4[key] for key in dataframes4.keys()],ignore_index=True)
#for i in range(11,33):
#    combine_df=pd.concat([combine_df,dataframes4['df'+str(i)]],axis=1)

merged_df = combine_df
merged_df['time'] = pd.to_datetime(merged_df['time'])
for i in range(len(df_collection)):
    print(i)
    df_collection[i]['time'] = pd.to_datetime(df_collection[i]['time'])
    merged_df = pd.merge_asof(merged_df,df_collection[i],on='time',direction='nearest')

0
1
2
3
4
5
6
7
8


In [37]:
merged_df

,Unnamed: 0.1,Unnamed: 0,空调光期温度设定,空调暗期温度设定,空调温度上限设定,空调温度下限设定,新风机手动开度,新风机手自动模式,空调自动模式,户外温度,...,plant 5,plant 5 diff,plant 6,plant 6 diff,plant 7,plant 7 diff,plant 8,plant 8 diff,plant 9,plant 9 diff
0,0,0,22.0,18.0,18.0,17.0,100.0,False,False,17.925346,...,107.47,NaN,NaN,NaN,162.56,NaN,137.80,NaN,139.15,NaN
1,1,1,22.0,18.0,18.0,17.0,100.0,False,False,17.170139,...,107.47,NaN,NaN,NaN,162.56,NaN,137.80,NaN,139.15,NaN
2,2,2,22.0,18.0,18.0,17.0,100.0,False,False,17.378471,...,107.47,NaN,NaN,NaN,162.56,NaN,137.80,NaN,139.15,NaN
3,3,3,22.0,18.0,18.0,17.0,100.0,False,False,17.635998,...,107.47,NaN,NaN,NaN,162.56,NaN,137.80,NaN,139.15,NaN
4,4,4,22.0,18.0,18.0,17.0,100.0,False,False,17.774883,...,107.47,NaN,NaN,NaN,162.56,NaN,137.80,NaN,139.15,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35716,35716,35716,24.5,21.6,19.5,18.5,0.0,False,True,24.652779,...,7.20,-0.3,NaN,NaN,224.53,0.02,8.65,-1.42,179.66,-5.86
35717,35717,35717,24.5,21.6,19.5,18.5,0.0,False,True,24.537041,...,7.20,-0.3,NaN,NaN,224.53,0.02,8.65,-1.42,179.66,-5.86
35718,35718,35718,24.5,21.6,19.5,18.5,0.0,False,True,24.635414,...,7.20,-0.3,NaN,NaN,224.53,0.02,8.65,-1.42,179.66,-5.86
35719,35719,35719,24.5,21.6,19.5,18.5,0.0,False,True,24.733795,...,7.20,-0.3,NaN,NaN,224.53,0.02,8.65,-1.42,179.66,-5.86
